In [1]:
!pip install pandas

  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/

In [3]:
import pandas as pd
import json
import os

# 현재 작업 디렉토리 기준으로 data 폴더 경로 설정
BASE_DIR = os.path.dirname(os.getcwd())
CSV_PATH = os.path.join(BASE_DIR, "data", "KC_PET_ACP_CTLSTT_LC_DATA_2023.csv")
JSON_PATH = os.path.join(BASE_DIR, "data", "pet_friendly_places_2023.json")

# 폴더가 없으면 생성
os.makedirs(os.path.dirname(JSON_PATH), exist_ok=True)

# CSV 파일 읽기
df = pd.read_csv(CSV_PATH, encoding="utf-8")

# 변환 결과 저장 리스트
documents = []

for _, row in df.iterrows():
    location = f"{row['CTPRVN_NM']} {row['SIGNGU_NM']}" if pd.notnull(row['SIGNGU_NM']) else row['CTPRVN_NM']
    content = (
        f"{location}에 위치한 '{row['FCLTY_NM']}'은(는) {row['FCLTY_INFO_DC']}입니다. "
        f"반려동물 동반은 {'가능' if row['PET_POSBL_AT'] == 'Y' else '불가능'}하며, "
        f"조건은 '{row['PET_INFO_CN']}'입니다. "
        f"주소는 도로명 '{row['RDNMADR_NM']}', 지번 '{row['LNM_ADDR']}'이고, "
        f"연락처는 '{row['TEL_NO'] if pd.notnull(row['TEL_NO']) else '정보 없음'}'입니다."
    )
    metadata = {
        "facility_name": row['FCLTY_NM'],
        "province": row['CTPRVN_NM'],
        "city": row['SIGNGU_NM'] if pd.notnull(row['SIGNGU_NM']) else "",
        "road_address": row['RDNMADR_NM'],
        "jibun_address": row['LNM_ADDR'],
        "latitude": row['LC_LA'],
        "longitude": row['LC_LO'],
        "phone": row['TEL_NO'],
        "category": row['FCLTY_INFO_DC'],
        "operation_time": row['OPER_TIME'],
        "closed_days": row['RSTDE_GUID_CN'],
        "pet_allowed": row['PET_POSBL_AT'],
        "pet_info": row['PET_INFO_CN'],
        "pet_size": row['ENTRN_POSBL_PET_SIZE_VALUE'],
        "pet_limit": row['PET_LMTT_MTR_CN'],
        "indoor_allowed": row['IN_PLACE_ACP_POSBL_AT'],
        "outdoor_allowed": row['OUT_PLACE_ACP_POSBL_AT'],
        "parking": row['PARKNG_POSBL_AT'],
        "price_info": row['UTILIIZA_PRC_CN'],
        "pet_additional_fee": row['PET_ACP_ADIT_CHRGE_VALUE']
    }
    documents.append({
        "content": content,
        "metadata": metadata
    })

# JSON 파일로 저장
with open(JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)

print(f"✅ JSON 변환 완료: {JSON_PATH}")

✅ JSON 변환 완료: c:\Users\treze\OneDrive\바탕 화면\3rd1st\data\pet_friendly_places_2023.json
